## **Scrapping Table from Wikipedia Page**

**Importing necessary libraries**

In [25]:

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import time
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

**Specifying the required URL/web page for scraping**

In [26]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

## **Requesting to get the url page and putting the HTML into the data variable**


**Using BeautifulSoup library we parse the HTML from our URL into the BeautifulSoup parse tree format and saving in 'soup' variable**

In [27]:
data= requests.get(url)
soup= BeautifulSoup(data.text,"lxml")

**Using the 'find_all' function to bring back all instances of the 'table' tag in the HTML and store in 'all_tables' variable**

In [28]:
all_tables=soup.find_all("table")
all_tables[0]

<table cellpadding="2" cellspacing="0" rules="all" style="width:100%; border-collapse:collapse; border:1px solid #ccc;">
<tbody><tr>
<td style="width:11%; vertical-align:top; color:#ccc;">
<p><b>M1A</b><br/><span style="font-size:80%;"><i>Not assigned</i></span>
</p>
</td>
<td style="width:11%; vertical-align:top; color:#ccc;">
<p><b>M2A</b><br/><span style="font-size:80%;"><i>Not assigned</i></span>
</p>
</td>
<td style="width:11%; vertical-align:top;">
<p><b>M3A</b><br/><span style="font-size:80%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>)</span>
</p>
</td>
<td style="width:11%; vertical-align:top;">
<p><b>M4A</b><br/><span style="font-size:80%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>)</span>
</p>
</td>
<td style="width:11%; vertical-align:top;">
<p><b>M5A</b><br/><span style="font-size:80%;"><a hr

**Checking if the table extracted is the correct one by verifying number of rows**

In [29]:
rows=all_tables[0].find_all('tr')
len(rows)


20

Looping through the rows to get the data. The table is well structured with 20 rows and 9 columns. I want to scrape the code, borough and neighbourhood. 
So, I set up two empty lists (A, B). By observing xml tree I found that Postal codes are the text value under 'b' tag and Borough, neighbourhood are the text value under 'span' tag. 

To start with, I used the Beautiful Soup ‘find_all’ function again and set it to look for the string ‘tr’. Then I set up a FOR loop for each row within that array and set Python to loop through the rows, one by one.

Within the loop I used find_all again to search each row for <td> tags with the ‘td’ string.I add all of these to a variable called ‘col’ and then check to make sure that there are 9 items in our ‘col’ array (i.e. one for each column and total 9 columns).

In [30]:
A=[]
B=[]
for row in all_tables[0].find_all('tr'):
  col= row.find_all('td')
  if len(col)==9:
    for i in range(9):
      col1=col[i].find('b')
      A.append(col1.text)
      col2=col[i].find('span')
      B.append(col2.text)
      


# Converting the extracted data in the list to a dataframe

**Dropping rows with Not Assigned entry in Borough(Neighbourhood) column**

In [31]:
df=pd.DataFrame(A,columns=['PostalCode'])
df['Borough(Neighbourhood)']=B
df=df[df['Borough(Neighbourhood)']!="Not assigned"]
df.head()

,PostalCode,Borough(Neighbourhood)
2,M3A,North York(Parkwoods)
3,M4A,North York(Victoria Village)
4,M5A,Downtown Toronto(Regent Park / Harbourfront)
5,M6A,North York(Lawrence Manor / Lawrence Heights)
6,M7A,Queen's Park / Ontario Provincial Government
8,M9A,Etobicoke(Islington Avenue)
9,M1B,Scarborough(Malvern / Rouge)
11,M3B,North York(Don Mills)North
12,M4B,East York(Parkview Hill / Woodbine Gardens)
13,M5B,"Downtown Toronto(Garden District, Ryerson)"


**Refining the dataframe to get in desired shape** 

In [32]:
df[['Borough','Neighbourhood']] = df['Borough(Neighbourhood)'].str.split('(',expand=True, n=1)
df.drop(['Borough(Neighbourhood)'], axis="columns", inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods)
3,M4A,North York,Victoria Village)
4,M5A,Downtown Toronto,Regent Park / Harbourfront)
5,M6A,North York,Lawrence Manor / Lawrence Heights)
6,M7A,Queen's Park / Ontario Provincial Government,None
8,M9A,Etobicoke,Islington Avenue)
9,M1B,Scarborough,Malvern / Rouge)
11,M3B,North York,Don Mills)North
12,M4B,East York,Parkview Hill / Woodbine Gardens)
13,M5B,Downtown Toronto,"Garden District, Ryerson)"


In [33]:
df['Neighbourhood']=df['Neighbourhood'].str.replace(')',"")
df['Neighbourhood']=df['Neighbourhood'].str.replace('/',",")

In [34]:
df.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Queen's Park / Ontario Provincial Government,None
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern , Rouge"
11,M3B,North York,Don MillsNorth
12,M4B,East York,"Parkview Hill , Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


**Checking if we have borough which has corresponding neighbourhood empty and copying the borough entry to the empty cell in the neighbourhood column**

In [35]:
df["Neighbourhood"].isnull().sum()

1

In [36]:
df.loc[df["Neighbourhood"].isnull(),'Neighbourhood'] = df["Borough"] 

In [37]:
df.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Queen's Park / Ontario Provincial Government,Queen's Park / Ontario Provincial Government


**Final shape of the dataframe obtained**

In [38]:
df.shape

(103, 3)

In [2]:
! pip install geocoder

     |████████████████████████████████| 102kB 3.6MB/s ta 0:00:01     |██████████                      | 30kB 3.1MB/s eta 0:00:01
     |████████████████████████████████| 829kB 3.0MB/s eta 0:00:01
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491095 sha256=a54db04aaba92be730c2d3ce3226c49d182f52f208b09d3ef8abbe93f9a125f2
  Stored in directory: /Users/yashbajoria/Library/Caches/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future


In [21]:
import geocoder # import geocoder

In [ ]:
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(df['PostalCode']))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

**Merging GeoSpatial Coordinates to the main dataset by PostalCode Variable**

In [39]:
df1=pd.read_csv('Geospatial_Coordinates.csv')

In [40]:
dataframe= df.merge(df1, how='inner', on='PostalCode', left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)

# Final Dataset 

In [42]:
dataframe.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park / Ontario Provincial Government,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
